# Variance Reduction Techniques for Monte Carlo Simulation (European Option Pricing)

This notebook features functions specifically designed to apply variance reduction techniques for the pricing of European Options.

The functions provided are as follows:
- MC_Euro(S, K, T, r, sigma, N, option_type): Estimates the price of a European option, where option_type indicates whether it is a call or a put option.
- MC_Euro_ant(S, K, T, r, sigma, N, option_type): Estimates the price of a European option using **Antithetic Variates**.
- MC_Euro_con(S, K, T, r, sigma, N, option_type): Estimates the price of a European option using **Control Variates**.
- MC_Euro_imp(S, K, T, r, sigma, N, option_type): Estimates the price of a European option using **Importance Sampling**.
- MC_Euro_str(S, K, T, r, sigma, N, option_type): Estimates the price of a European option using **Stratified Sampling**.

In [1]:
import numpy as np
from scipy.stats import norm

import matplotlib.pyplot as plt
import seaborn as sns

rng = np.random.default_rng(12345)

In [2]:
def MC_Euro(S, K, T, r, sigma, N, option_type='call'):
    
    if option_type not in ['call', 'put']:
        raise ValueError("Error: Invalid option type. Please choose 'call' or 'put'.")
    
    S0 = S
    X = rng.normal(0, 1, N)
    ST = S0 * np.exp((r - (0.5 * sigma**2)) * T + sigma * np.sqrt(T) * X)
    
    if option_type == 'call':
        fST = np.exp(-r*T)*np.maximum(ST-K,0)
    elif option_type == 'put':
        fST = np.exp(-r*T)*np.maximum(K-ST,0)
        
    price = np.mean(fST)
    variance = np.var(fST)
    
    return price, variance

def MC_Euro_ant(S, K, T, r, sigma, N, option_type='call'):
    
    if option_type not in ['call', 'put']:
        raise ValueError("Error: Invalid option type. Please choose 'call' or 'put'.")
    
    S0 = S
    X = rng.normal(0, 1, N)
    
    STu = S0 * np.exp((r - (0.5 * sigma**2)) * T + sigma * np.sqrt(T) * X)
    STd = S0 * np.exp((r - (0.5 * sigma**2)) * T - sigma * np.sqrt(T) * X)
    
    if option_type == 'call':
        fSTu = np.exp(-r*T)*np.maximum(STu-K,0)
        fSTd = np.exp(-r*T)*np.maximum(STd-K,0)
    elif option_type == 'put':
        fSTu = np.exp(-r*T)*np.maximum(K-STu,0)
        fSTd = np.exp(-r*T)*np.maximum(K-STd,0)
    
    Z = (fSTu + fSTd) / 2
    
    ant_price = np.mean(Z)
    ant_variance = np.var(Z)
    
    return ant_price, ant_variance

def MC_Euro_con(S, K, T, r, sigma, N, option_type='call'):
    
    if option_type not in ['call', 'put']:
        raise ValueError("Error: Invalid option type. Please choose 'call' or 'put'.")
    
    S0 = S
    
    mean_ST = S0 * np.exp(r * T)
    var_ST = mean_ST**2 * (np.exp(sigma**2 * T) - 1)
    
    X = rng.normal(0, 1, N)
    ST = S0 * np.exp((r - (0.5 * sigma**2)) * T + sigma * np.sqrt(T) * X)
    
    if option_type == 'call':
        fST = np.exp(-r*T) * np.maximum(ST-K,0)
    elif option_type == 'put':
        fST = np.exp(-r*T) * np.maximum(K-ST,0)
    
    cov = np.mean(fST * ST) - (np.mean(fST) * np.mean(ST))
    
    c = cov / var_ST
    
    f_c = fST - c * (ST - mean_ST)
    
    con_price = np.mean(f_c)
    con_variance = np.var(f_c)
    
    return con_price, con_variance

def MC_Euro_imp(S, K, T, r, sigma, N, option_type='call'):
    
    if option_type not in ['call', 'put']:
        raise ValueError("Error: Invalid option type. Please choose 'call' or 'put'.")
    
    S0 = S
    
    y_k = norm.cdf((np.log(K/S0) - (r-(0.5)*sigma**2) * T) / (sigma * np.sqrt(T)))
    
    if option_type == 'call':
        Y = rng.uniform(y_k, 1, N)
        X = norm.ppf(Y)
        ST = S0 * np.exp((r - (0.5 * sigma**2)) * T + sigma * np.sqrt(T) * X)
        fST = np.exp(-r * T) * (ST - K)
        g = (1 - y_k) * fST
    elif option_type == 'put':
        Y = rng.uniform(0, y_k, N)
        X = norm.ppf(Y)
        ST = S0 * np.exp((r - (0.5 * sigma**2)) * T + sigma * np.sqrt(T) * X)
        fST = np.exp(-r * T) * (K - ST)
        g = y_k * fST

    imp_price = np.mean(g)
    imp_variance = np.var(g)
    
    return imp_price, imp_variance

def MC_Euro_str(S, K, T, r, sigma, N, option_type='call'):
    if option_type not in ['call', 'put']:
        raise ValueError("Error: Invalid option type. Please choose 'call' or 'put'.")
    
    S0 = S
    
    M = 10
    N_strat = N // M
    
    if M * N_strat != N:
        raise ValueError("Error: M does not divide N")
    
    str_price = 0
    str_var = 0
    for m in range(M):
        Y = rng.uniform(m/M, (m+1)/M, N_strat)
        X = norm.ppf(Y)
        ST = S0 * np.exp((r - (0.5 * sigma**2)) * T + sigma * np.sqrt(T) * X)
        
        if option_type == 'call':
            fST = np.exp(-r*T) * np.maximum(ST-K,0)
        elif option_type == 'put':
            fST = np.exp(-r*T) * np.maximum(K-ST,0)
        
        str_price += np.mean(fST)
        str_var += np.var(fST)
        
    str_price /= M
    str_var /= M
    
    return str_price, str_var

In [3]:
def BS_call(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2 /2) * T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r*T) * norm.cdf(d2)

def BS_put(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2 /2) * T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return -S * norm.cdf(-d1) + K * np.exp(-r*T) * norm.cdf(-d2)

def BS_price(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2 /2) * T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    BS_call = S * norm.cdf(d1) - K * np.exp(-r*T) * norm.cdf(d2)
    BS_put = BS_call - S + K * np.exp(-r*T)
    return BS_call, BS_put

In [4]:
# Set parameters
S = 110
K = 110
T = 1.5
r = 0.03
sigma = 0.25
N = 10000

In [5]:
MC_price, MC_var = MC_Euro(S, K, T, r, sigma, N)
MC_price_ant, MC_var_ant = MC_Euro_ant(S, K, T, r, sigma, N)
MC_price_con, MC_var_con = MC_Euro_con(S, K, T, r, sigma, N)
MC_price_imp, MC_var_imp = MC_Euro_imp(S, K, T, r, sigma, N)
MC_price_str, MC_var_str = MC_Euro_str(S, K, T, r, sigma, N)

SEM_naive = np.sqrt(MC_var/N)
print("Naive MC price is ", '{0:.5g}'.format(MC_price), 
      "+/-", '{0:.2g}'.format(1.96*SEM_naive) )

SEM_ant = np.sqrt(MC_var_ant/N)
print("Anithetic MC price is ", '{0:.5g}'.format(MC_price_ant), 
      "+/-", '{0:.2g}'.format(1.96*SEM_ant) )

SEM_con = np.sqrt(MC_var_con/N)
print("Control MC price is ", '{0:.5g}'.format(MC_price_con),
      "+/-", '{0:.2g}'.format(1.96*SEM_con) )

SEM_imp = np.sqrt(MC_var_imp/N)
print("Importance MC price is ", '{0:.5g}'.format(MC_price_imp), 
      "+/-", '{0:.2g}'.format(1.96*SEM_imp) )

SEM_str = np.sqrt(MC_var_str/N)
print("Stratified MC price is ", '{0:.5g}'.format(MC_price_str), 
      "+/-", '{0:.2g}'.format(1.96*SEM_str) )

Naive MC price is  15.679 +/- 0.5
Anithetic MC price is  15.609 +/- 0.27
Control MC price is  15.635 +/- 0.19
Importance MC price is  15.891 +/- 0.28
Stratified MC price is  15.575 +/- 0.16


In [6]:
MC_price, MC_var = MC_Euro(S, K, T, r, sigma, N, 'put')
MC_price_ant, MC_var_ant = MC_Euro_ant(S, K, T, r, sigma, N, 'put')
MC_price_con, MC_var_con = MC_Euro_con(S, K, T, r, sigma, N, 'put')
MC_price_imp, MC_var_imp = MC_Euro_imp(S, K, T, r, sigma, N, 'put')
MC_price_str, MC_var_str = MC_Euro_str(S, K, T, r, sigma, N, 'put')

SEM = np.sqrt(MC_var/N)
print("Naive MC price is ", '{0:.5g}'.format(MC_price), 
      "+/-", '{0:.2g}'.format(1.96*SEM) )

SEM = np.sqrt(MC_var_ant/N)
print("Anithetic MC price is ", '{0:.5g}'.format(MC_price_ant), 
      "+/-", '{0:.2g}'.format(1.96*SEM) )

SEM = np.sqrt(MC_var_con/N)
print("Control MC price is ", '{0:.5g}'.format(MC_price_con),
      "+/-", '{0:.2g}'.format(1.96*SEM) )

SEM = np.sqrt(MC_var_imp/N)
print("Importance MC price is ", '{0:.5g}'.format(MC_price_imp), 
      "+/-", '{0:.2g}'.format(1.96*SEM) )

SEM = np.sqrt(MC_var_str/N)
print("Stratified MC price is ", '{0:.5g}'.format(MC_price_str), 
      "+/-", '{0:.2g}'.format(1.96*SEM) )

Naive MC price is  10.729 +/- 0.29
Anithetic MC price is  10.827 +/- 0.14
Control MC price is  10.852 +/- 0.19
Importance MC price is  10.814 +/- 0.14
Stratified MC price is  10.803 +/- 0.054


In [7]:
BS_call, BS_put = BS_price(S, K, T, r, sigma)

print("BS Call Price: ", round(BS_call, 4))
print("BS Put Price: ", round(BS_put, 4))

BS Call Price:  15.6499
BS Put Price:  10.8096
